In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

import gc
import warnings
warnings.filterwarnings('ignore')

In [3]:
%%time
# Load datasets using context managers
def load_datasets():
    try:
        # Load CSV files
        train=pd.read_csv("~/data/brist1d/train.csv")
        test=pd.read_csv("~/data/brist1d/test.csv")
        sample_submission=pd.read_csv("~/data/brist1d/sample_submission.csv")
        
        print("Datasets loaded successfully.")
        return train, test, sample_submission
    
    except Exception as e:
        print(f"Error loading datasets: {e}")
        return None
    
train, test, sample, = load_datasets()

Datasets loaded successfully.
CPU times: user 4.83 s, sys: 2.61 s, total: 7.45 s
Wall time: 7.46 s


In [4]:
# Display information for all datasets
# Display dataset information
def display_dataset_info(dataset, name):
    
    print("-----------------------------------------------------------------")

    print(f"{name} DataFrame Shape: Rows = {dataset.shape[0]}, Columns = {dataset.shape[1]}")
    
    total_missing = dataset.isnull().sum().sum()
    print(f"There are total {total_missing} missing values in the {name} DataFrame.") if total_missing > 0 else print(f"There are no missing values in the {name} DataFrame.")

    total_duplicates = dataset.duplicated().sum()
    print(f"There are {total_duplicates} duplicate rows in the {name} DataFrame.") if total_duplicates > 0 else print(f"There are no duplicate rows in the {name} DataFrame.")
    
    display(dataset.describe().style.set_caption(f"Descriptive Statistics for {name} Dataset").background_gradient(cmap="coolwarm"))
    print("-----------------------------------------------------------------")
    gc.collect()

datasets = [train, test, sample]
names = [ "Train", "Test", "Sample"]
for i in range(len(datasets)):
    display_dataset_info(datasets[i], names[i])

-----------------------------------------------------------------
Train DataFrame Shape: Rows = 177024, Columns = 508
There are total 40294794 missing values in the Train DataFrame.
There are no duplicate rows in the Train DataFrame.


-----------------------------------------------------------------
-----------------------------------------------------------------
Test DataFrame Shape: Rows = 3644, Columns = 507
There are total 862409 missing values in the Test DataFrame.
There are no duplicate rows in the Test DataFrame.


-----------------------------------------------------------------
-----------------------------------------------------------------
Sample DataFrame Shape: Rows = 3644, Columns = 2
There are no missing values in the Sample DataFrame.
There are no duplicate rows in the Sample DataFrame.


,bg+1:00
count,3644.000000
mean,8.665477
std,3.248307
min,2.200000
25%,6.300000
50%,8.000000
75%,10.300000
max,24.600000


-----------------------------------------------------------------


In [5]:
# Function to reduce memory usage
def reduce_memory(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage before optimization: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            if str(col_type).startswith("int"):
                df[col] = pd.to_numeric(df[col], downcast="integer")
            elif str(col_type).startswith("float"):
                df[col] = pd.to_numeric(df[col], downcast="float")
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage after optimization: {end_mem:.2f} MB")
    print(f"Reduced by {(100 * (start_mem - end_mem) / start_mem):.1f}%")

    return df

# Apply memory reduction
train = reduce_memory(train)
test = reduce_memory(test)

Memory usage before optimization: 686.10 MB
Memory usage after optimization: 311.18 MB
Reduced by 54.6%
Memory usage before optimization: 14.10 MB
Memory usage after optimization: 6.46 MB
Reduced by 54.2%


In [ ]:
%%time

# Encode categorical variables  
categorical_columns = train.select_dtypes(include="category").columns  

for col in categorical_columns:  
    # Initialize the LabelEncoder  
    le = LabelEncoder()  
    
    # Fit and transform the categorical column  
    train[col] = le.fit_transform(train[col])  
    test[col] = le.fit_transform(test[col])
    # Save the encoder for inverse transformation later  

In [7]:
train.head(20)

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.600000,NaN,NaN,9.700000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.700000,NaN,NaN,9.200000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.200000,NaN,NaN,8.700000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.700000,NaN,NaN,8.400000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.400000,NaN,NaN,8.100000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7
5,p01_5,p01,07:25:00,NaN,NaN,8.100000,NaN,NaN,8.300000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.4
6,p01_6,p01,07:40:00,NaN,NaN,8.300000,NaN,NaN,9.600000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.9
7,p01_7,p01,07:55:00,NaN,NaN,9.600000,NaN,NaN,11.100000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.1
8,p01_8,p01,08:10:00,NaN,NaN,11.100000,NaN,NaN,11.800000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.1
9,p01_9,p01,08:25:00,NaN,NaN,11.800000,NaN,NaN,12.800000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.9
